### BRAINTELS LABS 
*ARTIFICIAL INTELLIGENCE , IMAGE PROCESSING & EMBEDDED SYSTEMS*

In [3]:
import tensorflow as tf
import numpy as np
tf.compat.v1.disable_eager_execution()

### ProtoBuf GraphDef
El protocol buffer GraphDef es lo que tensorflow utiliza para definir sus grafos computacionles. con 

Con el conocimiento del **GraphDef** se puede reconstruir el grafo computacional que el protobuf representa, ademas se puede exportar e importar un GraphDef.

#### Creación de un GraphDef

el método GraphDef de un objeto de la clase **tf.Graph** contiene un método denominado as_graph_def() que retorna el grafo serializado denominado protocol buffer **GraphDef**.

#### Importar un GraphDef 

**tf.graph_util.import_graph_def(graph_def,input_map,name="import")** :

sirve para importar un protobuf GraphDef en un grafo computacional , de tal manera que extrae los elementos del **GraphDef** y los agrega al grafo computacional por default .


In [90]:
tf.compat.v1.reset_default_graph()

In [89]:
grafo=tf.Graph()
with grafo.as_default():
    x0=tf.compat.v1.placeholder(name="xo",dtype=tf.float32,shape=(3,3))
    x1=3*tf.ones(shape=(3,1),dtype=tf.float32,name="x1")
    x2=4*tf.ones(shape=(1,3),dtype=tf.float32,name="x2")
    y=tf.add(tf.matmul(x1,x2),x0,name="y")

#retornar el grafo serializado GraphDef    
gd=grafo.as_graph_def()
print("====== GRAFO GraphDef protobuf------------")

====== GRAFO GraphDef protobuf------------


### SavedModel  
El SavedModel representa  a un protocol Buffer SavedModel que contiene el programa completo de los meta grafos computacionales y sus variables que estan pueden tener definidas .

### construir un SavedModel

**tf.compat.v1.saved_model.Builder(export_dir)**:

construir un objeto de la clase SavedModel con el fin de guardar los grafos , variables y los assets.el argumento **export_dir** indica el directorio donde se almacenar estos elementos.
#### Método add_meta_graph_and_variables()

**add_meta_graph_and_variables(sesion,tags,signatures)**:

tiene como objetivo agregar los meta grafos y variables al modelo SavedModel . los meta-grafos estaran especificados por el argumento **tags**. 

los signatures representaran a las entradas y salidas especificas para el modelo.

la sesion es el argumento que indica que se requiere de una sesión para poder ejecutar dicho método.

#### Método save() 

**save(as_text)** 

tiene como objetivo almacenar en disco el protobuf SavedModel en el direcotiro export_dir . despues de utilizar este método se encontrara un archivo con nombre **saved_model.pb** donde la extensión **.pb** representa  un protocolBuffer en formato binario. 

 


In [59]:
export_path="brainpb"
#crear SavedModel protobuf
saved_model=tf.compat.v1.saved_model.Builder(export_path)
with tf.compat.v1.Session(graph=grafo) as sesion:
    inputs={"xin":x0}
    outputs={"y":y}
    #generar el signature inputs y outputs
    signa= tf.compat.v1.saved_model.predict_signature_def(inputs,outputs)
    #agregar metagrafos y variables al protobuf SavedModel
    saved_model.add_meta_graph_and_variables(sesion,["my_graph"],{"signas":signa})
    #almacenar en disco , ruta: "brainpb\saved_model.pb"
    saved_model.save() 

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: brainpb\saved_model.pb


In [91]:
tf.compat.v1.reset_default_graph()
#crear grafo computacional vacio
grafo_copy=tf.Graph()
with grafo_copy.as_default():
#conseguir el protobuf GraphDef y extraer sus elementos en el grafo por default grafo_copy
    tf.graph_util.import_graph_def(gd)
#convertir a un GraphDef para verificar que corresponden a las mismas operaciones
gc=grafo_copy.as_graph_def()
print(gc)
 

node {
  name: "import/xo"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 3
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "import/x1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 3
          }
          dim {
            size: 1
          }
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "import/mul/x"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 3.0
      }
    }
  }
}
node {
  name: "import/mul"
  op: "Mul"
  input: "import/mul/x"
  input: "import/x1"
  attr {
    key: 